In [72]:
import pandas as pd
import numpy as np
import pyodbc
import warnings
import datetime
from datetime import timedelta
import logging
from node_rules_new import cgcr_oxygen_tree, cgcr_aluminium_prediction, vag_aluminium_prediction

warnings.filterwarnings('ignore')

# for logging purposes
now = datetime.datetime.now()
print("Start Time : ", now)


####################################################################################

def call_prediction_function(row):

    """
    This function calls either the cgcr/vag prediction function based on the grade type for each heat
    """

    if row['GRADE_TYPE'] == 'CG/CR':
        return cgcr_aluminium_prediction(row)

    elif row['GRADE_TYPE'] == 'VAG':
        print("Grade type = VAG")
        return vag_aluminium_prediction(row)

    else:
        print("Unknown Grade")
        return None


def find_latest_heats(conn):
    """
    This function checks the SQL server and finds the list of heats for which AL Prediction is to be done.
    If this list is empty, then the predictive model is not run

    :param conn: SQL server connection client
    """
    # reading the table which has information regarding sample chemistry
    sample_chemistry_data= pd.read_sql("SELECT TOP(100) * FROM heat_analysis order by SAMPLE_TIME desc", conn)
    sample_chemistry_data= pd.read_sql("SELECT * FROM heat_analysis where HEAT_NUMBER in ('22200172','22400185','22200161') order by SAMPLE_TIME desc", conn)

    # keeping only data points from the last 3 hours
    # window selected based on analysis
    #sample_chemistry_data = sample_chemistry_data[sample_chemistry_data['SAMPLE_TIME']>= now - timedelta(minutes = 180)]

    # first we remove all records of LF5, as we are not predicting for that LF
    sample_chemistry_data = sample_chemistry_data[sample_chemistry_data['AGGREGATE'] != 'LF5']

    # for this, first we have to find the heats which have the latest sample information as first sample
    sample_latest = sample_chemistry_data.groupby('HEAT_NUMBER').agg({'SAMPLE_CODE':'max'})
    sample_latest = sample_latest[sample_latest['SAMPLE_CODE'].isin(['25101','25201'])]
    sample_latest.reset_index(inplace = True)

    # getting the latest heat numbers from master_output_table
    # we've already predicted the outputs for this heat, so we don't need to predict again
    previous_heats = pd.read_sql("SELECT TOP(100) HEAT_NUMBER FROM dbo.output_table ORDER BY MSG_TIME_STAMP desc",conn)
    previous_heats = list(previous_heats['HEAT_NUMBER'])
    previous_heats = [str(x) for x in previous_heats]

    # we will only run for any new heats that have been found
    latest_heats = sample_latest['HEAT_NUMBER'].unique()
    run_heats = [x for x in latest_heats if x not in previous_heats]

    return run_heats


Start Time :  2022-01-12 11:18:44.506410


In [73]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=STEELDNA;'
                     'Database=DeS;'
                     'UID=sa;'
                     'PWD=admin@123;'
                     'Trusted_Connection=no;')

cursor = conn.cursor()

######################### checking if model needs to be run #####################

# getting list of heats for which model has not yet been run
run_heats = find_latest_heats(conn)
print(run_heats)


['22200161', '22400185']


In [91]:
heat_analysis = pd.read_sql("select * from heat_analysis where HEAT_NUMBER in ('22200172','22400185','22200161') ", conn)
lf_heat_data = pd.read_sql("select * from lf_heat_data where HEAT_NUMBER in ('22200172','22400185','22200161')", conn)

grade_mapping = pd.read_sql("select * from grade_mapping", conn)


In [92]:
heat_analysis

,SEQ_VALUE,MSG_TIME_STAMP,MSG_FLAG,AGGREGATE,SAMPLE_TIME,SAMPLE_CODE,HEAT_NUMBER,C,SI,S,...,NIO,TIO2,TIFE,CAF2,FEO,V2O5,CO2,K2O,NA2O,B
0,8547,2022-01-10 23:09:08.397,N,LF1,2022-01-10 23:02:01,25201,22200161,0.026,0.003,0.028,...,None,None,None,None,None,None,None,None,None,0.0
1,8592,2022-01-11 11:59:09.110,N,LF2,2022-01-11 11:53:04,25101,22200172,0.040,0.005,0.020,...,None,None,None,None,None,None,None,None,None,0.0
2,8593,2022-01-11 12:17:09.100,N,LF2,2022-01-11 12:15:04,25201,22200172,0.049,0.026,0.006,...,None,None,None,None,None,None,None,None,None,0.0
3,8604,2022-01-11 15:29:09.273,N,LF4,2022-01-11 15:20:55,25101,22400185,0.034,0.000,0.023,...,None,None,None,None,None,None,None,None,None,0.0


In [93]:
lf_heat_data

,SEQ_VALUE,MSG_TIME_STAMP,MSG_FLAG,HEAT_NUMBER,GRADE_TYPE,LM_START_WT,TAP_O2,O2AFTERCELOX,LTA,FIRSTMEASTEMP,STATION,LIME,SIMN,ALBAR
0,31441880,2022-01-11 11:51:54.033,N,22200172,JDHCG04BCN,179.0,738.0,244.0,112.15,1576.0,2.0,294.0,None,0.0
1,31442169,2022-01-11 11:59:54.067,N,22200172,JDHST22EXZ,179.0,738.0,244.0,112.15,1576.0,2.0,294.0,None,396.0
2,31442634,2022-01-11 12:10:54.070,N,22200172,JDHST22EXZ,179.0,738.0,244.0,112.15,1576.0,2.0,294.0,None,396.0
3,31442850,2022-01-11 12:15:54.067,N,22200172,JDHST22EXZ,179.0,738.0,244.0,112.15,0.0,2.0,608.0,None,396.0
4,31442858,2022-01-11 12:15:54.093,N,22200172,JDHST22EXZ,179.0,738.0,244.0,112.15,0.0,2.0,608.0,None,396.0
5,31450125,2022-01-11 15:18:24.240,N,22400185,JDBC18HMMZ,189.0,1130.0,0.0,97.35,1598.0,4.0,895.0,None,0.0
6,31450515,2022-01-11 15:28:24.247,N,22400185,JDHST22EXZ,189.0,1130.0,0.0,97.35,1598.0,4.0,895.0,None,484.0
7,31450897,2022-01-11 15:38:24.270,N,22400185,JDHST22EXZ,189.0,1130.0,0.0,97.35,1598.0,4.0,895.0,None,484.0


In [94]:
# if not empty, model is to be run
#if len(run_heats) != 0:

    # importing heat_analysis
#     heat_analysis = pd.read_sql("select TOP(1000) * from heat_analysis order by MSG_TIME_STAMP desc", conn)
#     heat_analysis = heat_analysis[heat_analysis['HEAT_NUMBER'].isin(run_heats)]

#     # importing lf_heat_data
#     lf_heat_data = pd.read_sql("select TOP(1000) * from lf_heat_data order by MSG_TIME_STAMP desc", conn)
#     lf_heat_data = lf_heat_data[lf_heat_data['HEAT_NUMBER'].isin(run_heats)]

    # importing the grade mapping fact table
grade_mapping = pd.read_sql("select * from grade_mapping", conn)

######################### data processing ##############################

# calculating required columns
lf_heat_data['final_O2_ppm'] = lf_heat_data['TAP_O2'] + lf_heat_data['O2AFTERCELOX']

# converting ALBAR to tons
lf_heat_data['ALBAR'] = lf_heat_data['ALBAR']/19

# renaming the required columns
lf_heat_data.rename(columns = {
    'GRADE_TYPE':'GRADE',
    'LM_START_WT':'LM_Start_Wt_lf',
    'FIRSTMEASTEMP':'1st_Probe_temp_lf',
    'ALBAR':'Al_Bar_lf',
    'LIME':'Lime_tap',
    'SIMN':'SiMn_tap',
    'LTA':'LTA_lf'
}, inplace = True)

# keeping only the required rows
lf_heat_data = lf_heat_data.groupby(['HEAT_NUMBER','STATION']).agg({'MSG_TIME_STAMP':'max','GRADE': 'first','1st_Probe_temp_lf':'max', 'Al_Bar_lf':'max','LM_Start_Wt_lf':'max','LTA_lf':'max','Lime_tap':'max','SiMn_tap':'max','final_O2_ppm':'max'})
lf_heat_data.reset_index(inplace = True)
lf_heat_data.sort_values(by = ['MSG_TIME_STAMP'], ascending = False, inplace = True)
lf_heat_data.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)

lf_heat_data['GRADE'].replace(
    to_replace=['JDHST22EXZ','JDBC18HMMZ'],
    value='JDHCG04BCN',
    inplace=True
)

# preprocessing heat_analysis
# if there are multiple records for a heat, keeping only the latest one
#heat_analysis.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)
heat_analysis.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)

# keeping only required columns
heat_analysis = heat_analysis[['HEAT_NUMBER','AGGREGATE','AL_TOTAL','S','C','SI']]

# renaming columns as per requirement
heat_analysis.rename(columns = {'AL_TOTAL':'AL_chem_first', 'S':'S_chem_first','SI':'SI_chem_first','C':'C_chem_first'}, inplace = True)


# merging sample chemistry data with lf data
ads = heat_analysis.merge(lf_heat_data, on = ['HEAT_NUMBER'], how = 'inner')
print(len(ads))
# filling nulls with zeros
# these will get clipped in the next step
ads.fillna(0, inplace = True)

# ads preprocessing

# constraints to avoid cases with data issues
# constraint no.1
# al_bar range is 15-25
ads['Al_Bar_lf'] = ads['Al_Bar_lf'].clip(15,25)
# constraint no.2
# lm weight is between 170 - 205
ads['LM_Start_Wt_lf'] = ads['LM_Start_Wt_lf'].clip(170,205)

# these are the caps for each column
# TODO: add SiMn capping as well
ads['LTA_lf'] = ads['LTA_lf'].clip(60)
ads['Lime_tap'] = 294#ads['Lime_tap'].clip(500)
ads['1st_Probe_temp_lf'] = ads['1st_Probe_temp_lf'].clip(1550)
ads['final_O2_ppm'] = ads['final_O2_ppm'].clip(750)

# adding loi perc as a column
# this is a fixed value
ads['LOI_perc'] = 6

# mapping the grades to their grade type (CG/CR or VAG)
# the grades that don't get mapped are billet heats, and so will not be run in this model
    




2


In [95]:
ads

,HEAT_NUMBER,AGGREGATE,AL_chem_first,S_chem_first,C_chem_first,SI_chem_first,STATION,MSG_TIME_STAMP,GRADE,1st_Probe_temp_lf,Al_Bar_lf,LM_Start_Wt_lf,LTA_lf,Lime_tap,SiMn_tap,final_O2_ppm,LOI_perc
0,22200172,LF2,0.110,0.020,0.040,0.005,2.0,2022-01-11 12:15:54.093,JDHCG04BCN,1576.0,20.842105,179.0,112.15,294,0.0,982.0,6
1,22400185,LF4,0.018,0.023,0.034,0.000,4.0,2022-01-11 15:38:24.270,JDHCG04BCN,1598.0,25.000000,189.0,97.35,294,0.0,1130.0,6


In [96]:
ads.merge(grade_mapping, on = ['GRADE'], how = 'inner')

,HEAT_NUMBER,AGGREGATE,AL_chem_first,S_chem_first,C_chem_first,SI_chem_first,STATION,MSG_TIME_STAMP,GRADE,1st_Probe_temp_lf,Al_Bar_lf,LM_Start_Wt_lf,LTA_lf,Lime_tap,SiMn_tap,final_O2_ppm,LOI_perc,GRADE_TYPE
0,22200172,LF2,0.110,0.020,0.040,0.005,2.0,2022-01-11 12:15:54.093,JDHCG04BCN,1576.0,20.842105,179.0,112.15,294,0.0,982.0,6,CG/CR
1,22400185,LF4,0.018,0.023,0.034,0.000,4.0,2022-01-11 15:38:24.270,JDHCG04BCN,1598.0,25.000000,189.0,97.35,294,0.0,1130.0,6,CG/CR


In [97]:
ads = ads.merge(grade_mapping, on = ['GRADE'], how = 'inner')
ads

,HEAT_NUMBER,AGGREGATE,AL_chem_first,S_chem_first,C_chem_first,SI_chem_first,STATION,MSG_TIME_STAMP,GRADE,1st_Probe_temp_lf,Al_Bar_lf,LM_Start_Wt_lf,LTA_lf,Lime_tap,SiMn_tap,final_O2_ppm,LOI_perc,GRADE_TYPE
0,22200172,LF2,0.110,0.020,0.040,0.005,2.0,2022-01-11 12:15:54.093,JDHCG04BCN,1576.0,20.842105,179.0,112.15,294,0.0,982.0,6,CG/CR
1,22400185,LF4,0.018,0.023,0.034,0.000,4.0,2022-01-11 15:38:24.270,JDHCG04BCN,1598.0,25.000000,189.0,97.35,294,0.0,1130.0,6,CG/CR


In [98]:
ads.apply(call_prediction_function, axis = 1)

0    [86.32840863157894, 1, 1]
1          [265.2694255, 7, 4]
dtype: object

In [82]:
################ AL Prediction #################################

# dont predict if ads is empty

# predicting aluminium material
ads['al_predictions'] =  ads.apply(call_prediction_function, axis = 1)

print(ads['al_predictions'])

# splitting the list of outputs into separate columns
ads['AL_KGS'] = ads['al_predictions'].apply(lambda x: x[0])
ads['Carbon Node'] = ads['al_predictions'].apply(lambda x: x[1])
ads['Oxygen Node'] = ads['al_predictions'].apply(lambda x:x[2])


del ads['al_predictions']

ads['AL_KGS'][ads['AL_KGS']<0]=0

# predicting aluminium material in meters
ads['AL_MTS'] = ads['AL_KGS'] * (3)


# keeping only required columns
#ads = ads[['HEAT_NUMBER','AGGREGATE','AL_KGS','AL_MTS']]

0    [77.25380863157893, 1, 1]
1          [262.8654255, 7, 4]
Name: al_predictions, dtype: object


In [29]:
ads['AL_KGS']

0    77.253809
Name: AL_KGS, dtype: float64